In [1]:
import os

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/uravfarooqui3/mydatascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "uravfarooqui3"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "b0d319c375e913619d4cc710d2b92de39f4f525d"

In [7]:
import os
%cd d:/SEM VIII/dsproject_in_production/mydatascienceproject
print(os.getcwd())

d:\SEM VIII\dsproject_in_production\mydatascienceproject
d:\SEM VIII\dsproject_in_production\mydatascienceproject


d:\SEM VIII\dsproject_in_production\mydatascienceproject\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
%pwd

'd:\\SEM VIII\\dsproject_in_production\\mydatascienceproject'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN


        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/uravfarooqui3/mydatascienceproject.mlflow"
        )
        return model_evaluation_config

In [12]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib
import numpy as np

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [17]:
try:
    config_manager=ConfigurationManager()
    model_evaluation_config=config_manager.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    print(f"Error occurred: {e}")
    raise e

[2025-02-16 18:17:05,297]: [INFO]: [datasciencelogger]: [read_yaml]: [30]: [yaml file: config\config.yaml loaded successfully]
[2025-02-16 18:17:05,299]: [INFO]: [datasciencelogger]: [read_yaml]: [30]: [yaml file: params.yaml loaded successfully]
[2025-02-16 18:17:05,304]: [INFO]: [datasciencelogger]: [read_yaml]: [30]: [yaml file: schema.yaml loaded successfully]
[2025-02-16 18:17:05,306]: [INFO]: [datasciencelogger]: [create_directories]: [50]: [created directory at: artifacts]
[2025-02-16 18:17:05,307]: [INFO]: [datasciencelogger]: [create_directories]: [50]: [created directory at: artifacts/model_evaluation]
[2025-02-16 18:17:05,705]: [INFO]: [datasciencelogger]: [save_json]: [63]: [json file saved at: artifacts\model_evaluation\metrics.json]


2025/02/16 18:17:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/02/16 18:17:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run dapper-seal-5 at: https://dagshub.com/uravfarooqui3/mydatascienceproject.mlflow/#/experiments/0/runs/961e1ec1adad4fb2bcea89ff483503d2
🧪 View experiment at: https://dagshub.com/uravfarooqui3/mydatascienceproject.mlflow/#/experiments/0
